# Library Import

In [ ]:
import pandas as pd
import requests

# Web Scraping

source: https://coinmarketcap.com/pt-br/currencies/bitcoin/historical-data/

## Request API with requests library

This site have a API that returns the table with the historical price of Bitcoin since april 28, 2013. I use this API with the requests library and convert into JSON.

In [ ]:
url = 'https://api.coinmarketcap.com/data-api/v3/cryptocurrency/historical?id=1&convertId=2783&timeStart=1225452521&timeEnd=1636374521'
r = requests.get(url)
json_r = r.json()

In [ ]:
json_r['data']['quotes']

[{'quote': {'close': 268.3394874234,
   'high': 271.8784034574,
   'low': 264.1207522031,
   'marketCap': 2976240316.4600987,
   'open': 270.5188261014,
   'timestamp': '2013-04-28T23:59:59.999Z',
   'volume': 0.0},
  'timeClose': '2013-04-28T23:59:59.999Z',
  'timeHigh': '2013-04-28T18:50:02.000Z',
  'timeLow': '2013-04-28T20:15:02.000Z',
  'timeOpen': '2013-04-28T00:00:00.000Z'},
 {'quote': {'close': 288.993262576,
   'high': 294.8875203793,
   'low': 267.9195999997,
   'marketCap': 3206575467.67812,
   'open': 268.8073340879,
   'timestamp': '2013-04-29T23:59:59.999Z',
   'volume': 0.0},
  'timeClose': '2013-04-29T23:59:59.999Z',
  'timeHigh': '2013-04-29T13:15:01.000Z',
  'timeLow': '2013-04-29T05:20:01.000Z',
  'timeOpen': '2013-04-29T00:00:00.000Z'},
 {'quote': {'close': 279.3622000002,
   'high': 295.29989928,
   'low': 269.4136961336,
   'marketCap': 3100745818.6270847,
   'open': 289.4112000002,
   'timestamp': '2013-04-30T23:59:59.999Z',
   'volume': 0.0},
  'timeClose': '201

# Working with Pandas

In [ ]:
# Converting the JSON into a Pandas DataFrame
df = pd.json_normalize(json_r['data']['quotes'])
df.head()

,timeOpen,timeClose,timeHigh,timeLow,quote.open,quote.high,quote.low,quote.close,quote.volume,quote.marketCap,quote.timestamp
0,2013-04-28T00:00:00.000Z,2013-04-28T23:59:59.999Z,2013-04-28T18:50:02.000Z,2013-04-28T20:15:02.000Z,270.518826,271.878403,264.120752,268.339487,0.0,2.976240e+09,2013-04-28T23:59:59.999Z
1,2013-04-29T00:00:00.000Z,2013-04-29T23:59:59.999Z,2013-04-29T13:15:01.000Z,2013-04-29T05:20:01.000Z,268.807334,294.887520,267.919600,288.993263,0.0,3.206575e+09,2013-04-29T23:59:59.999Z
2,2013-04-30T00:00:00.000Z,2013-04-30T23:59:59.999Z,2013-04-30T08:25:02.000Z,2013-04-30T18:55:01.000Z,289.411200,295.299899,269.413696,279.362200,0.0,3.100746e+09,2013-04-30T23:59:59.999Z
3,2013-05-01T00:00:00.000Z,2013-05-01T23:59:59.999Z,2013-05-01T00:15:01.000Z,2013-05-01T19:55:01.000Z,278.528200,280.311581,215.849338,234.424558,0.0,2.602845e+09,2013-05-01T23:59:59.999Z
4,2013-05-02T00:00:00.000Z,2013-05-02T23:59:59.999Z,2013-05-02T14:25:01.000Z,2013-05-02T14:30:02.000Z,233.202238,251.677277,184.914468,210.819796,0.0,2.341475e+09,2013-05-02T23:59:59.999Z


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3117 entries, 0 to 3116
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   timeOpen         3117 non-null   object 
 1   timeClose        3117 non-null   object 
 2   timeHigh         3117 non-null   object 
 3   timeLow          3117 non-null   object 
 4   quote.open       3117 non-null   float64
 5   quote.high       3117 non-null   float64
 6   quote.low        3117 non-null   float64
 7   quote.close      3117 non-null   float64
 8   quote.volume     3117 non-null   float64
 9   quote.marketCap  3117 non-null   float64
 10  quote.timestamp  3117 non-null   object 
dtypes: float64(6), object(5)
memory usage: 268.0+ KB


In [ ]:
# Converting object data into Pandas datetime
df['timeHigh'] = pd.to_datetime(df['timeHigh']).dt.time
df['timeLow'] = pd.to_datetime(df['timeLow']).dt.time
df['quote.timestamp'] = pd.to_datetime(df['quote.timestamp']).dt.date

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3117 entries, 0 to 3116
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   timeOpen         3117 non-null   object 
 1   timeClose        3117 non-null   object 
 2   timeHigh         3117 non-null   object 
 3   timeLow          3117 non-null   object 
 4   quote.open       3117 non-null   float64
 5   quote.high       3117 non-null   float64
 6   quote.low        3117 non-null   float64
 7   quote.close      3117 non-null   float64
 8   quote.volume     3117 non-null   float64
 9   quote.marketCap  3117 non-null   float64
 10  quote.timestamp  3117 non-null   object 
dtypes: float64(6), object(5)
memory usage: 268.0+ KB


# Data Cleaning

## Requirements

In [ ]:
# Removing the timeOpen and the timeClose columns
df.drop('timeOpen', axis=1, inplace=True)
df.drop('timeClose', axis=1, inplace=True)
df.head()

,timeHigh,timeLow,quote.open,quote.high,quote.low,quote.close,quote.volume,quote.marketCap,quote.timestamp
0,18:50:02,20:15:02,270.518826,271.878403,264.120752,268.339487,0.0,2.976240e+09,2013-04-28
1,13:15:01,05:20:01,268.807334,294.887520,267.919600,288.993263,0.0,3.206575e+09,2013-04-29
2,08:25:02,18:55:01,289.411200,295.299899,269.413696,279.362200,0.0,3.100746e+09,2013-04-30
3,00:15:01,19:55:01,278.528200,280.311581,215.849338,234.424558,0.0,2.602845e+09,2013-05-01
4,14:25:01,14:30:02,233.202238,251.677277,184.914468,210.819796,0.0,2.341475e+09,2013-05-02


In [ ]:
# Renaming some columns
rename_cols = {'timeHigh':'time_higher_price', 
               'timeLow':'time_lower_price', 
               'quote.open':'opening_price', 
               'quote.high':'higher_price', 
               'quote.low':'lower_price', 
               'quote.close':'closing_price', 
               'quote.volume':'volume', 
               'quote.marketCap':'market_cap', 
               'quote.timestamp':'timestamp'}
df.rename(columns=rename_cols, inplace=True)
df.head()

,time_higher_price,time_lower_price,opening_price,higher_price,lower_price,closing_price,volume,market_cap,timestamp
0,18:50:02,20:15:02,270.518826,271.878403,264.120752,268.339487,0.0,2.976240e+09,2013-04-28
1,13:15:01,05:20:01,268.807334,294.887520,267.919600,288.993263,0.0,3.206575e+09,2013-04-29
2,08:25:02,18:55:01,289.411200,295.299899,269.413696,279.362200,0.0,3.100746e+09,2013-04-30
3,00:15:01,19:55:01,278.528200,280.311581,215.849338,234.424558,0.0,2.602845e+09,2013-05-01
4,14:25:01,14:30:02,233.202238,251.677277,184.914468,210.819796,0.0,2.341475e+09,2013-05-02


In [ ]:
# Organizing the Columns
col_order =df[['timestamp', 'time_higher_price', 'time_lower_price', 'opening_price', 'higher_price', 'lower_price', 'closing_price', 'volume', 'market_cap']]
df = col_order
df.head()

,timestamp,time_higher_price,time_lower_price,opening_price,higher_price,lower_price,closing_price,volume,market_cap
0,2013-04-28,18:50:02,20:15:02,270.518826,271.878403,264.120752,268.339487,0.0,2.976240e+09
1,2013-04-29,13:15:01,05:20:01,268.807334,294.887520,267.919600,288.993263,0.0,3.206575e+09
2,2013-04-30,08:25:02,18:55:01,289.411200,295.299899,269.413696,279.362200,0.0,3.100746e+09
3,2013-05-01,00:15:01,19:55:01,278.528200,280.311581,215.849338,234.424558,0.0,2.602845e+09
4,2013-05-02,14:25:01,14:30:02,233.202238,251.677277,184.914468,210.819796,0.0,2.341475e+09


In [ ]:
df.to_csv('/content/drive/MyDrive/Acadêmico/Data Science/Notebooks/Projects/Projetos de Estudo/Cryptocurrency/bitcoin_DataFrame.csv', index=False)